# The null space

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import matplotlib

matplotlib.rcParams["figure.figsize"] = (12, 4)

A key principle in the problem of mapping the surfaces of stars and planets is the idea of a *null space*. The null space of a (linear) transformation is the set of input vectors that result in a zero vector as output. In the context of mapping, the null space comprises the spherical harmonics (or combinations of spherical harmonics) that do not affect the observed flux whatsoever.

A trivial example is the $Y_{1,-1}$ spherical harmonic, which does not project into the light curve for rotations about the vertical ($\hat{y}$) axis:

In [ ]:
import starry
import numpy as np

starry.config.lazy = False
starry.config.quiet = True

map = starry.Map(1)
map[1, -1] = 1
map.show(theta=np.linspace(0, 360, 50))

It is clear that as this object rotates, the total flux does not change (and in fact is exactly zero). That's because the $Y_{1,-1}$ harmonic is perfectly symmetric under such rotations.

It might be hard to think of other harmonics that behave this way, but in fact the **vast majority** of spherical harmonics are usually in the null space of the light curve problem. Let's take a deeper look at this. First, we'll load `starry` and generate a high resolution map of the Earth.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import starry

starry.config.lazy = False
starry.config.quiet = True

In [ ]:
map1 = starry.Map(30)
map1.load("earth", sigma=0.05)
map1.show(projection="rect")

Let's now compute the light curve of this map as we rotate it one full cycle about the $\hat{y}$ direction:

In [ ]:
theta = np.linspace(0, 360, 1000)
flux1 = map1.flux(theta=theta)
plt.figure(figsize=(12, 5))
plt.plot(theta, flux1)
plt.xlabel(r"$\theta$ [degrees]", fontsize=24)
plt.ylabel("Flux [arbitrary units]", fontsize=24);

At phase 0, the prime meridian is facing the observer, and the flux drops as the Atlantic and then Pacific oceans come into view. The flux then peaks when Asia is in view, and that's the light curve of the Earth.

Now, let's create a new, identical map of the Earth:

In [ ]:
map2 = starry.Map(30)
map2.load("earth", sigma=0.05)

But this time, we'll zero out all coefficients corresponding to odd degrees above 2...

In [ ]:
for l in range(3, map2.ydeg + 1, 2):
    map2[l, :] = 0

... as well as all coefficients corresponding to negative values of `m`:

In [ ]:
for l in range(1, map2.ydeg + 1):
    map2[l, -l:0] = 0

Most of the coefficients are now zero:

In [ ]:
print(
    "Fractional size of the null space: %.3f"
    % (1 - np.count_nonzero(map2.y) / len(map2.y))
)

And here's what this silly map looks like:

In [ ]:
map2.show(projection="rect")

It doesn't really look anything like the Earth (though a sharp eye might spot the outline of Africa and some other familiar features -- barely). If you're wondering why we did this, let's plot the light curve of this new map next to the light curve of the original map of the Earth:

In [ ]:
flux2 = map2.flux(theta=theta)
plt.figure(figsize=(12, 5))
plt.plot(theta, flux1, label="map1")
plt.plot(theta, flux2, "--", label="map2")
plt.legend()
plt.xlabel(r"$\theta$ [degrees]", fontsize=24)
plt.ylabel("Flux [arbitrary units]", fontsize=24);

Their light curves are **identical**. Even though the maps *look* completely different, the actual differences between the two maps lie entirely in the null space (by construction). This means we have no way of distinguishing between these two maps if all we have access to is the light curve.

To drive this point home, here's what the two maps look like side-by-side:

In [ ]:
# HACK: Manually render both maps so that they are in sync

image1 = map1.render(theta=np.linspace(0, 360, 50, endpoint=False))
image2 = map2.render(theta=np.linspace(0, 360, 50, endpoint=False))

from matplotlib.animation import FuncAnimation
from IPython.display import HTML

fig, ax = plt.subplots(1, 2, figsize=(9, 3))

img1 = ax[0].imshow(image1[0], origin="lower", cmap="plasma", extent=(-1, 1, -1, 1))
img2 = ax[1].imshow(image2[0], origin="lower", cmap="plasma", extent=(-1, 1, -1, 1))

for i, axis in enumerate(ax):
    axis.axis("off")


def updatefig(i):
    img1.set_array(image1[i])
    img2.set_array(image2[i])
    return (img1, img2)


ani = FuncAnimation(fig, updatefig, interval=75, blit=True, frames=image1.shape[0],)

plt.close()
display(HTML(ani.to_html5_video()))

The total amount of flux at any given time -- equal to the brightness integrated over the entire disk -- is the same in both cases, even though the surfaces look nothing alike.

For objects rotating along an axis perpendicular to the line of sight (as in the example above), the null space consists of all of the $m < 0$ harmonics, as well as all of the harmonics of degree $l = 3, 5, 7 ...$:

In [ ]:
ydeg = 5
fig, ax = plt.subplots(ydeg + 1, 2 * ydeg + 1, figsize=(12, 6))
fig.subplots_adjust(hspace=0)
for axis in ax.flatten():
    axis.set_xticks([])
    axis.set_yticks([])
    axis.spines["top"].set_visible(False)
    axis.spines["right"].set_visible(False)
    axis.spines["bottom"].set_visible(False)
    axis.spines["left"].set_visible(False)
for l in range(ydeg + 1):
    ax[l, 0].set_ylabel(
        r"$l = %d$" % l,
        rotation="horizontal",
        labelpad=20,
        y=0.38,
        fontsize=10,
        alpha=0.5,
    )
for j, m in enumerate(range(-ydeg, ydeg + 1)):
    if m < 0:
        ax[-1, j].set_xlabel(
            r"$m {=} \mathrm{-}%d$" % -m, labelpad=10, fontsize=10, alpha=0.5
        )
    else:
        ax[-1, j].set_xlabel(r"$m = %d$" % m, labelpad=10, fontsize=10, alpha=0.5)

# Loop over the orders and degrees
map = starry.Map(ydeg=ydeg, quiet=True)
for i, l in enumerate(range(ydeg + 1)):
    for j, m in enumerate(range(-l, l + 1)):

        # Null space or not?
        if (m < 0) or (l == 3) or (l == 5):
            alpha = 0.25
        else:
            alpha = 1

        # Offset the index for centered plotting
        j += ydeg - l

        # Compute the spherical harmonic
        # with no rotation
        map.reset()
        if l > 0:
            map[l, m] = 1

        # Plot the spherical harmonic
        ax[i, j].imshow(
            map.render(),
            cmap="plasma",
            interpolation="none",
            origin="lower",
            extent=(-1, 1, -1, 1),
            alpha=alpha,
        )
        ax[i, j].set_xlim(-1.1, 1.1)
        ax[i, j].set_ylim(-1.1, 1.1)

The null space for this problem is indicated by the translucent harmonics above. (Note that in general certain linear combinations of the remaining harmonics are *also* in the null space, so the problem is actually *worse* than in the example above).

## The size of the null space

Because the light curve model in ``starry`` is linear, it's actually easy to compute the size of the null space for a given problem. Consider how the flux $\mathbf{f}$ is computed in ``starry``:

$$
\mathbf{f} = \mathbf{A} \mathbf{x}
$$

where $\mathbf{A}$ is the design matrix (which you can obtain from ``map.design_matrix()``) and $\mathbf{x}$ is just the (amplitude-weighted) spherical harmonic coefficient vector. When solving this system for $\mathbf{x}$, the matrix we end up needing to invert looks like

$$
\mathbf{I} = \mathbf{A}^\top \Sigma^{-1} \mathbf{A}
$$

where $\Sigma$ is the data covariance matrix. The object $\mathbf{I}$ is the *Fisher information matrix* and quantifies the maximum amount of information we can learn given our observations $\mathbf{f}$. While there's a lot we can do with $\mathbf{I}$, the most important thing in our case is the size of the null space, which is *equal to the number of rows (or columns) minus the rank of this matrix.*

There's lots of ways to think about why this is the case. Note that $\mathbf{I}$ is invertible if (and only if) it is full rank, meaning the null space is zero. That's the condition for a well-posed problem, in which we can learn everything there is to know about $\mathbf{x}$ from our observations $\mathbf{f}$. In nearly all cases in the mapping problem, $\mathbf{I}$ is *not* full rank, meaning it has linearly dependent rows or columns. For each such column, that's one fewer term (or combination of terms) we are able to constrain in $\mathbf{x}$. In other words, the size of the null space is equal to the number of linearly dependent rows/columns in $\mathbf{I}$.

Note, importantly, that in the limit that our measurements are uncorrelated, the rank of $\mathbf{I}$ is equal to the rank of $\mathbf{A}^\top \mathbf{A}$ (i.e., independent of the uncertainties). Note also that this quantity does not depend on either $\mathbf{f}$ *or* $\mathbf{x}$ -- just the design matrix!

Let's take a look at the size of the null space in the problem above. We'll use ``numpy.linalg.matrix_rank`` to compute the matrix rank.

In [ ]:
def print_info(A):
    """Print some information about the null space of the design matrix A."""
    # Get the Fisher information
    I = A.T.dot(A)

    # Compute its rank
    rank = np.linalg.matrix_rank(I)

    # Subtract off the size of the matrix
    N = I.shape[0]
    null_size = N - rank

    # Print
    print("Number of coefficients:         {}".format(N))
    print("Fisher rank:                    {}".format(rank))
    print("Size of null space:             {}".format(null_size))
    print("Fractional size of null space:  {:.3f}".format(null_size / N))

In [ ]:
A = map1.design_matrix(theta=theta)
print_info(A)

Our problem has 961 free coefficients, but the rank of the Fisher matrix iss *only* 33. That means that 928 of the coefficients (or, more precisely, 928 linear combinations of coefficients) lie in the null space. In other words, **over 96 percent of the information about the map cannot be probed from the light curve in this case.**

Yep. 96 percent.

## So, how can we possibly map other stars and planets?

Coming soon :)